In [ ]:
# G drive connection
# learn: to access a shared file just put a shortcut in mi drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# env set up

import pandas as pd

In [ ]:
# Verify que read esas vergas

!ls '/content/drive/MyDrive/andresrokp/Thesis Juandavid/'

In [ ]:
# Hola mundo a una de esas vergas

file_path = '/content/drive/MyDrive/andresrokp/Thesis Juandavid/Companies.dta'
df = pd.read_stata(file_path)
print('\n---\ninfo\n---')
df.info()  # Show column
print('\n---\ndescribe\n---')
print(df.describe())  # Show very basic stadistics
print('\n---\nhead\n---')
df.head()  # Show first rows


In [ ]:
# 1er modelo:

# traer tablas
file_path = '/content/drive/MyDrive/andresrokp/Thesis Juandavid/Investments.dta'
file_path_2 = '/content/drive/MyDrive/andresrokp/Thesis Juandavid/Investors.dta'

df_Investments_bnt = pd.read_stata(file_path)
print( 'df_Investments_bnt.info()', df_Investments_bnt.info() )
print( 'len (df_Investments)', len(df_Investments_bnt) )
df_Investors = pd.read_stata(file_path_2)
print( 'df_Investors.info()', df_Investors.info() )
print( 'len (df_Investors)', len(df_Investors) )

# Left JOIN de las tablas según InvestorID
merged_investments_investors = pd.merge(df_Investments_bnt, df_Investors, on='InvestorID', how='left')
# print(merged_investments_investors.head(5))

print('-\n--\n-')

# cleaning dejar solo VCs
df_Investments_merge_clean = merged_investments_investors[merged_investments_investors['InvestorType'] != 'BA']
df_Investments_merge_clean = df_Investments_merge_clean[df_Investments_merge_clean['InvestorType'] != 'Other']
df_Investments_merge_clean = df_Investments_merge_clean[~df_Investments_merge_clean['InvestorID'].str.contains('missing')]
print( 'df_Investments_merge_clean.info()', df_Investments_merge_clean.info() )
print( 'len (df_Investments_merge_clean)', len(df_Investments_merge_clean) )
# print(df_Investments_no_BAs.head(5))


col_InvestorID_clean = df_Investments_merge_clean['InvestorID']
# col_InvestorID_clean = pd.merge(df_Investments_bnt, df_Investors, on='InvestorID', how='left')['InvestorName']
# print(df_Investments_clean.head(5))

numberOf_Unique_InvestorsClean = len(col_InvestorID_clean.unique())
print( 'numberOf_Unique_InvestorsClean', numberOf_Unique_InvestorsClean )

uniqueInvestorsAppereanceCount = col_InvestorID_clean.value_counts()
print('uniqueInvestorsAppereanceCount\n', uniqueInvestorsAppereanceCount)
print( 'len( uniqueInvestorsAppereanceCount )', len( uniqueInvestorsAppereanceCount ) )
print( 'uniqueInvestorsAppereanceCount', uniqueInvestorsAppereanceCount )


print("\n\nInvestorName_Final conteo de apariciones TOP 25%")
col_InvestorID_clean.value_counts().head(round(numberOf_Unique_InvestorsClean*0.25))

# print('\n---\ninfo\n---')
# df_Investors_Reputation.info()  # Show column
# print('\n---\ndescribe\n---')
# print(df_Investors_Reputation.describe())  # Show very basic stadistics
# print('\n---\nhead\n---')
# df_Investors_Reputation.head()  # Show first rows
# print("\n\nInvestorName_Final missing_values")
# print( col_investor_name.isnull().sum() )

# print("\n\nInvestorName_Final unique")
# print( col_investor_name.unique() )

In [ ]:
# Exploración de tipo VC

file_path = '/content/drive/MyDrive/andresrokp/Thesis Juandavid/Investors.dta'
df_Investors = pd.read_stata(file_path)

print('\n---\ninfo\n---')
df_Investors.info()  # Show column
print('\n---\ndescribe\n---')
print(df_Investors.describe())  # Show very basic stadistics
print('\n---\nhead\n---')
df_Investors.head()  # Show first rows

col_investor_name = df_Investors['InvestorType']


# print("\n\nInvestorName_Final missing_values")
# print( col_investor_name.isnull().sum() )

# print("\n\nInvestorName_Final unique inversores")
# print( col_investor_name.unique() )
# numberOfUniqueInvestors = len(col_investor_name.unique())
# print( numberOfUniqueInvestors )

# esto es para comparar loque dice el pdf en cantidad de BA's y VC'c
print("\n\nInvestorName_Final conteo de apariciones tipo de inversor")
print( len(col_investor_name.value_counts() ) )
print( col_investor_name.value_counts() )